<a href="https://colab.research.google.com/github/JonatanSiracusa/download-historical-series/blob/main/download_hist_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualization Data Series


In this notebook we will be downloading the historical series of a list of stocks.

1. Byma´s prices downloaded from Yahoo Finance. 


In order to get the desired results, the next steps must be followed:

1. Open the Excel file named 'tickers.xlsx' located in the same folder of this program: 
	* Complete the `'ticker_byma'` column.
	* Complete the `'ticker_yahoo'` column. 
2. Set the `'start_date'` variable in the section 1 of this program.
3. Set the `'NOMBRE_OUTPUT'` variable in the section 1 of this program. Data series will be saved and named by the value set in this variable.


The next steps will be followed in order to implement the ***Project***:

1. Kick-off: Libraries Importing, Variables Setup and Functions.

2. Data Loading

3. Visualization

4. Performance Metrics


***************************



# 1. Kick-off: Libraries Importing, Variables Setup and Functions

In [ ]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
from fitter import Fitter
import math
import random
import time
import datetime as dt
from datetime import datetime

import yfinance as yf

#import matplotlib.pyplot as plt
#from matplotlib.ticker import FuncFormatter
#import seaborn as sns
#sns.set_theme(style='darkgrid')
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

import warnings

In [2]:
# medimos el tiempo de ejecucion del programa
star_time = time.time()

#start_date = dt.datetime(1994, 1, 1)
start_date = dt.datetime(2000, 8, 17)
end_date = dt.datetime.now()

RUEDAS_ANIO = 252
NOMBRE_INPUT = 'historical-Adj_prices_plus-byma'
EXPORTAR_DATOS = False

warnings.simplefilter("ignore")

In [ ]:
def get_volatility(ticker, df):
	"""
	Esta funcion busca en el DF y devuelve la volatilidad de las ultimas 40 ruedas anualizada del ticker ingresado, la cual tiene el sufijo '_v40'.
	"""
	variable = ticker +'_v40'
	valor = df.loc[:, variable].iloc[-1]
	return valor


# 2. Data Loading

In [6]:
tickers = ['^MERV', 'GGAL.BA', 'BBAR.BA', 'BMA.BA', 'SUPV.BA', 'VALO.BA']
start_date = dt.datetime(2023, 12, 11)
end_date = dt.datetime.now()

raw_data = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
prices_v1 = raw_data.copy()

prices_v1.index = prices_v1.index.strftime('%Y-%m-%d')
prices_v1.index = pd.to_datetime(prices_v1.index)

print('Index type:', prices_v1.index.dtype)
prices_v1

Index type: datetime64[ns]


Ticker,BBAR.BA,BMA.BA,GGAL.BA,SUPV.BA,VALO.BA,^MERV
Date,,,,,,
2023-12-11,1575.508667,2482.076660,1608.694946,756.297546,196.943512,976823.0
2023-12-12,1558.793579,2429.306885,1631.135376,736.449463,209.178940,1010022.0
2023-12-13,1586.386108,2538.740967,1673.680420,781.396606,211.531906,1003484.0
2023-12-14,1561.565430,2579.023682,1638.462769,770.268250,213.178986,989696.0
2023-12-15,1433.345703,2414.044189,1513.575684,703.257019,204.943604,925658.0
...,...,...,...,...,...,...
2025-01-29,8760.000000,11925.000000,8090.000000,3985.000000,439.500000,2545356.0
2025-01-30,8950.000000,12100.000000,8210.000000,4025.000000,436.000000,2599329.0
2025-01-31,8770.000000,11950.000000,8080.000000,3960.000000,428.500000,2564659.0


In [8]:
returns = np.log(prices_v1).diff().dropna()
returns

Ticker,BBAR.BA,BMA.BA,GGAL.BA,SUPV.BA,VALO.BA,^MERV
Date,,,,,,
2023-12-12,-0.010666,-0.021490,0.013853,-0.026594,0.060273,0.033422
2023-12-13,0.017546,0.044062,0.025749,0.059242,0.011186,-0.006494
2023-12-14,-0.015770,0.015743,-0.021267,-0.014344,0.007756,-0.013835
2023-12-15,-0.085677,-0.066107,-0.079284,-0.091016,-0.039397,-0.066893
2023-12-18,0.002546,-0.044859,-0.003516,-0.006667,-0.027941,-0.004093
...,...,...,...,...,...,...
2025-01-29,0.073406,0.051624,0.040359,0.055462,0.000000,0.044065
2025-01-30,0.021458,0.014568,0.014724,0.009988,-0.007995,0.020983
2025-01-31,-0.020317,-0.012474,-0.015961,-0.016281,-0.017352,-0.013428


In [4]:
# Loading tickers
xlsx = pd.ExcelFile('./tickers.xlsx')
df1 = pd.read_excel(xlsx, 'Hoja1')

tickers = df1.iloc[:, 0].tolist()

df1.info()
print(tickers)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ticker_byma   21 non-null     object
 1   ticker_yahoo  21 non-null     object
dtypes: object(2)
memory usage: 468.0+ bytes
['ALUA', 'BBAR', 'BMA', 'BYMA', 'CEPU', 'COME', 'CRES', 'CVH', 'EDN', 'GGAL', 'LOMA', 'MIRG', 'PAMP', 'SUPV', 'TECO2', 'TGNO4', 'TGSU2', 'TRAN', 'TXAR', 'VALO', 'YPFD']


In [5]:
# Loading price series
xlsx = pd.ExcelFile('./' + NOMBRE_INPUT + '.xlsx')
raw_df_prices = pd.read_excel(xlsx, sheet_name='Sheet1', index_col=0)#, usecols=[0, 1, 38, 39, 40, 41])  # Inds
prices_v1 = raw_df_prices.copy()

prices_v1.info()
print(get_volatility('GGAL', prices_v1))
prices_v1

<class 'pandas.core.frame.DataFrame'>
Index: 6997 entries, 1996-10-08 to 2024-11-01
Data columns (total 89 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   n          6997 non-null   int64  
 1   Index      6997 non-null   int64  
 2   Index_sr   6997 non-null   float64
 3   Index_lr   6997 non-null   float64
 4   Index_v40  6997 non-null   float64
 5   ALUA       6997 non-null   float64
 6   ALUA_sr    6997 non-null   float64
 7   ALUA_lr    6997 non-null   float64
 8   ALUA_v40   6997 non-null   float64
 9   BBAR       6997 non-null   float64
 10  BBAR_sr    6997 non-null   float64
 11  BBAR_lr    6997 non-null   float64
 12  BBAR_v40   6997 non-null   float64
 13  BMA        6997 non-null   float64
 14  BMA_sr     6997 non-null   float64
 15  BMA_lr     6997 non-null   float64
 16  BMA_v40    6997 non-null   float64
 17  BYMA       6997 non-null   float64
 18  BYMA_sr    6997 non-null   float64
 19  BYMA_lr    6997 non-null   float64
 20

,n,Index,Index_sr,Index_lr,Index_v40,ALUA,ALUA_sr,ALUA_lr,ALUA_v40,BBAR,...,TXAR_lr,TXAR_v40,VALO,VALO_sr,VALO_lr,VALO_v40,YPFD,YPFD_sr,YPFD_lr,YPFD_v40
Date,,,,,,,,,,,,,,,,,,,,,
1996-10-08,1,590,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1996-10-09,2,583,-0.011864,-0.011935,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1996-10-10,3,585,0.003431,0.003425,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1996-10-11,4,584,-0.001709,-0.001711,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1996-10-14,5,584,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-28,6993,1851914,-0.011144,-0.011207,0.273568,826.0,0.011016,0.010956,0.279871,5320.0,...,-0.017880,0.311983,332.0,0.001508,0.001507,0.225398,29600.0,-0.011686,-0.011755,0.349865
2024-10-29,6994,1845639,-0.003388,-0.003394,0.269149,832.0,0.007264,0.007238,0.280222,5310.0,...,0.012804,0.313617,333.5,0.004518,0.004508,0.224529,28700.0,-0.030405,-0.030877,0.343947
2024-10-30,6995,1837535,-0.004391,-0.004401,0.267646,826.0,-0.007212,-0.007238,0.277318,5220.0,...,0.003810,0.302848,331.0,-0.007496,-0.007524,0.218619,28450.0,-0.008711,-0.008749,0.327964


# 3. Visualization

In [6]:
"""
prices_v2 en base 100


prices_v3 returns y volats  

"""

'\nprices_v2 en base 100\n\n\nprices_v3 returns y volats  \n\n'

## 3.1. Visualization processing data

Definimos las Functions que vamos a estar usando durante las visualizaciones.

In [ ]:
# Creamos el DF que vamos a graficar
def cols_no_vacias(inicio, df):
	fila = series_graf.loc[indice_inicio]
	tickers_cotizantes = fila.index[fila != 0]
	return tickers_cotizantes.to_list()


def generar_series_graf(inicio=None, activos=None, calculos=None, x_values=False):
	"""
	Toma el DF prices_v1 y devuelve la variacion por 100 respecto al inicio de la serie de los activos y calculos solicitados, de un periodo.
	Args: 
		inicio (str): fecha en formato YYYY-MM-DD.
		activos (str): lista de tickers.
		calculos (str): lista de calculos. Puede contener uno, varios o ninguno.
			* 'p' (default): precio
			* 'sr': simple return
			* 'lr': log return
			* 'v40': volatilidad 40 ruedas
		x_values (bool): 
			* True: agrega una col con los mismos datos del index.
			* False (default): no agrega nada.
	Returns: 
		serie (Pandas DF): datos solicitados en formato Pandas DF.
	"""
	inicio = start_date.strftime('%Y-%m-%d') if inicio is None else inicio
	activos = tickers if activos is None else activos
	calculos = ['p'] if calculos is None else calculos

	mask = (prices_v1.index >= inicio)
	serie = prices_v1.loc[mask]
	serie.drop('n', axis=1, inplace=True)
		
	lista_tickers = []
	for activo in activos:
		for calculo in calculos:
			if calculo == 'sr':
				lista_tickers.append(activo + '_sr')
			elif calculo == 'lr':
				lista_tickers.append(activo + '_lr')
			elif calculo == 'v40':
				lista_tickers.append(activo + '_v40')
			else:
				lista_tickers.append(activo)

	# seleccionamos solo los tickers cotizan al momento de indice_inicio
	serie = serie[lista_tickers]

	# agregamos una col con los mismos datos del index
	if x_values:
		serie['x_values'] = serie.index
		serie.insert(0, 'x_values', serie.pop('x_values'))
	
	return serie


def normalize_prices(activos, df):
	"""
	Normaliza los datos de los activos solicitados en el DataFrame, para que todos tengan la misma base y puedan ser comparados.
	Devuelve la variacion por 100 respecto al inicio de la serie de los activos.
	"""
	# eliminar la columna x_value y luego de normalizar volver a agregarla
	flag = False
	if 'x_values' in df.columns:
		df = df.drop(columns='x_values')
		flag = True

	for activo in activos:
		#print(df.loc[:, activo].iloc[0] * 1 - 1)
		df[activo] = (df[activo] / df[activo].iloc[0] * 1 - 1)
		df.fillna(0, inplace=True)
		
	if flag:
		df['x_values'] = df.index
		df.insert(0, 'x_values', df.pop('x_values'))
	
	return df


def crear_etiqueta_mes(meses, df):
	"""
	Toma el DF especificado y devuelve el primer dia habil de cada mes, que se encuentra en el Index del DF.
	Args:
		meses (lista): es el numero correspondiente al mes que queremos tener como etiqueta
		df (Pandas DatatFrame): es el DF a partir del cual queremos tomar las etiquetas
	Returns:
		anio_mes (lista): lista con el primer dia habil de cada mes buscado, en formato str YYYY-MM-DD.
	"""
	# convertimos los indices en fechas y creamos un DF
	fechas = pd.to_datetime(df.index.to_list())
	df_fechas = pd.DataFrame({'fecha': fechas})

	# extraemos anio y mes
	df_fechas['anio'] = df_fechas['fecha'].dt.year
	df_fechas['mes'] = df_fechas['fecha'].dt.month

	# filtramos las fechas
	df_fechas_filtrado = df_fechas[df_fechas['mes'].isin(meses)]
	# seleccionamos el primer dia habil de los meses buscados
	df_fechas_filtrado = df_fechas_filtrado.groupby(['anio', 'mes']).fecha.min().reset_index()

	# formateamos a string y convertimos en una lista
	df_fechas_filtrado['fecha'] = df_fechas_filtrado['fecha'].dt.strftime('%Y-%m-%d')
	anio_mes = df_fechas_filtrado['fecha'].tolist()

	return anio_mes


## 3.2. Visualizations

### 3.2.1. Graph 1

In [8]:
inicio = '2023-12-11'
activos = ['Index', 'GGAL', 'BBAR', 'BMA', 'SUPV', 'VALO']
meses = [i for i in range(13)]

fecha_inicio = datetime.strptime(inicio, '%Y-%m-%d').strftime('%d/%m/%Y')
fecha_fin = datetime.now().strftime('%d/%m/%Y')

series_graf_1 = generar_series_graf(inicio, activos)
series_graf_1 = normalize_prices(activos, series_graf_1)
eje_x_etiquetas = crear_etiqueta_mes(meses, series_graf_1)
eje_x_titulo = f'Fechas respecto de cotizaciones diarias (período {fecha_inicio}-{fecha_fin})'

In [ ]:
fig = go.Figure()

for col in series_graf_1.columns[1:]:
	fig.add_trace(go.Scatter(
		x=series_graf_1.index,
		y=series_graf_1[col],
		mode='lines',
		#text=['punto1', 'punto2'],
		#textposition='bottom right',
		#line=dict(color='blue', width=3, dash='dash'),
		line=dict(width=2),
		#marker=dict(color='red', size=8, symbol='circle', line=dict(color='black', width=2)),
		opacity=0.8,
		#fill='tozeroy',
		#fill='tonexty',
		#fillcolor='rgba(0, 100, 250, 0.2)',
		name=col
	))

fig.add_trace(go.Scatter(
		x=series_graf_1.index,
		y=series_graf_1['Index'],
		mode='lines',
		line=dict(color='blue', width=3, dash='dot'),
		opacity=0.8,
		name='Index'
	))

fig.update_layout(
	width=1000, 
	height=600,
	title=dict(text='Rendimiento principales activos bancarios', x=0.5, y=0.95, font=dict(size=24)),
 
	xaxis=dict(
		title=dict(
			text=eje_x_titulo,
			font=dict(size=15),
			standoff=20
		),
		tickangle=45, tickfont=dict(size=12), tickvals=eje_x_etiquetas, ticktext=eje_x_etiquetas
	),
	yaxis=dict(
		title=dict(
			text='Variacion por 100 respecto al inicio de la serie', 
			font=dict(size=15),
			standoff=20
		),
		type='linear', side='right', tickfont=dict(size=13)
	),
	legend=dict(
		title='Activos', x=0.02, y=0.98, font=dict(size=12), bgcolor='rgba(255, 255, 255, 0.5)'
	),
	margin=dict(l=40, r=40, t=70, b=40)
)

fig.show()

### 3.2.2. Graph 2

In [10]:
inicio = '2023-12-11'
activos = ['GGAL', 'BBAR', 'BMA', 'VALO']
calculos=['p', 'sr', 'v40']
x_values=True
meses = [i for i in range(13)]

fecha_inicio = datetime.strptime(inicio, '%Y-%m-%d').strftime('%d/%m/%Y')
fecha_fin = datetime.now().strftime('%d/%m/%Y')

series_graf_2 = generar_series_graf(inicio, activos, calculos, x_values)
series_graf_2 = normalize_prices(activos, series_graf_2)
eje_x_etiquetas = crear_etiqueta_mes(meses, series_graf_2)
eje_x_titulo = f'Fechas respecto de cotizaciones diarias (período {fecha_inicio}-{fecha_fin})'

In [ ]:
# Crear la cuadrícula de subgráficos: 2 filas x 2 columnas
fig = make_subplots(
	rows=2, cols=2, 
	shared_xaxes=False,  # Ejes x compartidos
	shared_yaxes=True,  # Ejes y compartidos
	vertical_spacing=0.1,  # Espacio vertical entre subgráficos
	horizontal_spacing=0.1,  # Espacio horizontal entre subgráficos
	subplot_titles=("GGAL", "BBAR", "BMA", "VALO")  # Títulos de cada subgráfico
)

# Añadir el primer gráfico (Scatter) en la posición (1,1)
for col in series_graf_2.columns[1:4]:
	fig.add_trace(go.Scatter(
		x=series_graf_2.index,
		y=series_graf_2[col],
		mode='lines',
		#line=dict(color='blue', width=3, dash='dash'),
		line=dict(width=2),
		opacity=0.8,
		name=col),
		row=1, col=1
	)

# Añadir el segundo gráfico en la posición (1,2)
for col in series_graf_2.columns[4:7]:
	fig.add_trace(go.Scatter(
		x=series_graf_2.index,
		y=series_graf_2[col],
		mode='lines',
		#line=dict(color='blue', width=3, dash='dash'),
		line=dict(width=2),
		opacity=0.8,
		name=col),
		row=1, col=2
	)

# Añadir el tercer gráfico en la posición (2,1)
for col in series_graf_2.columns[7:10]:
	fig.add_trace(go.Scatter(
		x=series_graf_2.index,
		y=series_graf_2[col],
		mode='lines',
		#line=dict(color='blue', width=3, dash='dash'),
		line=dict(width=2),
		opacity=0.8,
		name=col),
		row=2, col=1
	)

# Añadir el cuarto gráfico en la posición (2,2)
for col in series_graf_2.columns[10:13]:
	fig.add_trace(go.Scatter(
		x=series_graf_2.index,
		y=series_graf_2[col],
		mode='lines',
		#line=dict(color='blue', width=3, dash='dash'),
		line=dict(width=2),
		opacity=0.8,
		name=col),
		row=2, col=2
	)

fig.update_layout(
	width=1100, 
	height=1400,
	title=dict(text='Rendimiento principales activos bancarios', x=0.5, y=0.97, font=dict(size=24), pad=dict(t=10)),
	#annotations=[
	#    dict(
	#        text=eje_x_titulo,
	#        x=0.5,                  # Centrado horizontalmente
	#        y=-0.09,                 # Debajo de los subgráficos
	#        xref="paper",           # Usa las coordenadas del layout global
	#        yref="paper",
	#        font=dict(size=16)      # Tamaño de la fuente del título
	#    )
	#],

	#xaxis=dict(
	#	title=dict(
	#		text=eje_x_titulo,
	#		font=dict(size=12),
	#		standoff=15
	#	),
	#	tickangle=45, tickfont=dict(size=12), tickvals=eje_x_etiquetas, ticktext=eje_x_etiquetas
	#),
	#xaxis=dict(tickangle=45, tickfont=dict(size=12), tickvals=eje_x_etiquetas, ticktext=eje_x_etiquetas
	#),

	yaxis=dict(
		title=dict(
			text='Variacion por 100 respecto al inicio de la serie', 
			font=dict(size=12),
			standoff=20
		),
		type='linear', side='right', tickfont=dict(size=13)
	),
	legend=dict(
		title='Activos',
		bgcolor='rgba(255, 255, 255, 0.5)',
		orientation="h",      # Orientación horizontal
		x=0.5,                # Centrada horizontalmente
		xanchor="center",
		y=-0.10,               # Ubicada debajo de los subgráficos
		yanchor="top",
		#title_text=None,      # Sin título
		traceorder="normal",
		itemsizing="constant",
		font=dict(size=12),
		tracegroupgap=0,      # Ajusta el espaciado entre grupos
	),
	legend_tracegroupgap=20,
	margin=dict(l=40, r=60, t=120, b=100)
)

# Configuración de ejes personalizados
#fig.update_xaxes(title_text="Eje X (común)", row=2, col=1)

# Aplicar las mismas etiquetas de valores en el eje x a todos los subgráficos
for i in range(1, 3): # Para filas 1 y 2
	for j in range(1, 3): # Para columnas 1 y 2
		fig.update_xaxes(
			tickangle=45, 
			tickfont=dict(size=12),
			tickvals=eje_x_etiquetas,           # Define los valores del eje x
			ticktext=eje_x_etiquetas,      # Define las etiquetas personalizadas para estos valores
			row=i, col=j
		)

#fig.update_xaxes(title_text="Eje X (común)", row=1, col=1)
#fig.update_yaxes(title_text="Variacion por 100", row=1, col=1)

for i in range(1, 3): # Para filas 1 y 2
	for j in range(1, 3): # Para columnas 1 y 2
		fig.update_yaxes(
			#tickangle=45, 
			#tickfont=dict(size=12),
			#tickvals=eje_x_etiquetas,           # Define los valores del eje x
			#ticktext=eje_x_etiquetas,      # Define las etiquetas personalizadas para estos valores
			#row=i, col=j,

			title=dict(
				text='Variacion por 100 respecto al inicio de la serie', font=dict(size=14), standoff=15
				), type='linear', side='right', tickfont=dict(size=13),	row=i, col=j
		)

#fig.update_yaxes(title=dict(
#	text='Variacion por 100 respecto al inicio de la serie', font=dict(size=14), standoff=15
#	), type='linear', side='right', tickfont=dict(size=13),	row=1, col=1)
#fig.update_yaxes(title=dict(
#	text='Variacion por 100 respecto al inicio de la serie', font=dict(size=14), standoff=15
#	), type='linear', side='right', tickfont=dict(size=13),	row=1, col=2)
#fig.update_yaxes(title=dict(
#	text='Variacion por 100 respecto al inicio de la serie', font=dict(size=14), standoff=15
#	), type='linear', side='right', tickfont=dict(size=13),	row=2, col=1)
#fig.update_yaxes(title=dict(
#	text='Variacion por 100 respecto al inicio de la serie', font=dict(size=14), standoff=15
#	), type='linear', side='right', tickfont=dict(size=13),	row=2, col=2)
#fig.update_yaxes(title_text="Eje Y (Serie 2)", row=1, col=2)
#fig.update_yaxes(title_text="Eje Y (Serie 3)", row=2, col=1)
#fig.update_yaxes(title_text="Eje Y (Serie 4)", row=2, col=2)
#showlegend=True


fig.show()

In [ ]:
"""# Crear la cuadrícula de subgráficos: 2 filas x 2 columnas
fig = make_subplots(
	rows=2, cols=2, 
	subplot_titles=("Gráfico 1", "Gráfico 2", "Gráfico 3", "Gráfico 4")  # Títulos de cada subgráfico
)

# Datos de ejemplo para cada subgráfico
x_values = [1, 2, 3, 4, 5]
y_values1 = [10, 20, 30, 40, 50]
y_values2 = [50, 40, 30, 20, 10]
y_values3 = [15, 25, 35, 45, 55]
y_values4 = [5, 15, 25, 35, 45]

# Añadir el primer gráfico (Scatter) en la posición (1,1)
fig.add_trace(
	go.Scatter(x=x_values, y=y_values1, mode="lines+markers", name="Serie 1"), 
	row=1, col=1
)

# Añadir el segundo gráfico (Bar) en la posición (1,2)
fig.add_trace(
	go.Bar(x=x_values, y=y_values2, name="Serie 2"), 
	row=1, col=2
)

# Añadir el tercer gráfico (Scatter) en la posición (2,1)
fig.add_trace(
	go.Scatter(x=x_values, y=y_values3, mode="lines", name="Serie 3"), 
	row=2, col=1
)

# Añadir el cuarto gráfico (Bar) en la posición (2,2)
fig.add_trace(
	go.Bar(x=x_values, y=y_values4, name="Serie 4"), 
	row=2, col=2
)

# Configurar el layout general de la figura
fig.update_layout(
	title="Figura con Subgráficos",
	showlegend=True  # Mostrar leyenda compartida en la figura
)



fig.add_trace(go.Scatter(
		x=series_graf.index,
		y=series_graf[activos[0]],
		mode='lines',
		line=dict(color='blue', width=3, dash='dot'),
		opacity=0.8,
		name='Index'
	))


# Configuración de ejes personalizados
#fig.update_xaxes(title_text="Eje X (común)", row=2, col=1)
#fig.update_yaxes(title_text="Variacion por 100", row=1, col=1)
#fig.update_yaxes(title_text="Eje Y (Serie 2)", row=1, col=2)
#fig.update_yaxes(title_text="Eje Y (Serie 3)", row=2, col=1)
#fig.update_yaxes(title_text="Eje Y (Serie 4)", row=2, col=2)
#showlegend=True

# Mostrar el gráfico
fig.show()"""


"""
# Crear subgráficos con ejes compartidos y espaciado personalizado
fig = make_subplots(
	rows=2, cols=2, 
	shared_xaxes=True,  # Ejes x compartidos
	shared_yaxes=True,  # Ejes y compartidos
	vertical_spacing=0.1,  # Espacio vertical entre subgráficos
	horizontal_spacing=0.05,  # Espacio horizontal entre subgráficos
	subplot_titles=("Gráfico 1", "Gráfico 2", "Gráfico 3", "Gráfico 4")
)"""

'\n# Crear subgráficos con ejes compartidos y espaciado personalizado\nfig = make_subplots(\n    rows=2, cols=2, \n    shared_xaxes=True,  # Ejes x compartidos\n    shared_yaxes=True,  # Ejes y compartidos\n    vertical_spacing=0.1,  # Espacio vertical entre subgráficos\n    horizontal_spacing=0.05,  # Espacio horizontal entre subgráficos\n    subplot_titles=("Gráfico 1", "Gráfico 2", "Gráfico 3", "Gráfico 4")\n)'

In [ ]:
"""
# Datos de ejemplo
x_values = [1, 2, 3, 4, 5]  # Valores comunes en el eje x
series_graf_2.columns[0]

custom_labels = ["A", "B", "C", "D", "E"]  # Etiquetas personalizadas para el eje x
eje_x_etiquetas

y_values1 = [10, 20, 30, 40, 50]
y_values2 = [50, 40, 30, 20, 10]


# Aplicar las mismas etiquetas de valores en el eje x a todos los subgráficos
for i in range(1, 3): # Para filas 1 y 2
	for j in range(1, 3): # Para columnas 1 y 2
		fig.update_xaxes(
			tickvals=x_values,           # Define los valores del eje x
			ticktext=custom_labels,      # Define las etiquetas personalizadas para estos valores
			row=i, col=j
		)"""

'\n# Datos de ejemplo\nx_values = [1, 2, 3, 4, 5]  # Valores comunes en el eje x\nseries_graf_2.columns[0]\n\ncustom_labels = ["A", "B", "C", "D", "E"]  # Etiquetas personalizadas para el eje x\neje_x_etiquetas\n\ny_values1 = [10, 20, 30, 40, 50]\ny_values2 = [50, 40, 30, 20, 10]\n\n\n# Aplicar las mismas etiquetas de valores en el eje x a todos los subgráficos\nfor i in range(1, 3): # Para filas 1 y 2\n    for j in range(1, 3): # Para columnas 1 y 2\n        fig.update_xaxes(\n            tickvals=x_values,           # Define los valores del eje x\n            ticktext=custom_labels,      # Define las etiquetas personalizadas para estos valores\n            row=i, col=j\n        )'

# 4. Performance metrics

In [14]:
ahora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
end_time = time.time()
execution_time = end_time - star_time

print(f'Correctly executed. Date: {ahora}.')
print(f'\nExecution time: {round(execution_time, 2)} seconds.')

Correctly executed. Date: 2024-11-06 12:10:44.

Execution time: 5.82 seconds.
